# just topbar

In [6]:
import bpy

class TextImport(bpy.types.Operator):
    bl_idname = "curve.text_import"
    bl_label = "Import Text/Markdown"
    
    # Let users pick only .txt or .md in the File Browser
    filepath: bpy.props.StringProperty(
        subtype='FILE_PATH', 
        options={'SKIP_SAVE'},
        default="",
        name="File Path"
    )
    filter_glob: bpy.props.StringProperty(
        default="*.txt;*.md",
        options={'HIDDEN'}
    )

    @classmethod
    def poll(cls, context):
        return context.area is not None

    def execute(self, context):
        # Check if we have a valid file path and extension
        if not self.filepath or not self.filepath.lower().endswith((".md", ".txt")):
            self.report({'WARNING'}, "Please select a .txt or .md file.")
            return {'CANCELLED'}

        # Read the file contents
        with open(self.filepath, 'r', encoding='utf-8') as file:
            text_content = file.read()

        # Create a new Text Curve
        text_curve = bpy.data.curves.new(name="Text", type="FONT")
        text_curve.body = text_content
        
        # Create a new object for the text
        text_object = bpy.data.objects.new(name="Text", object_data=text_curve)
        
        # Link object to current scene
        context.scene.collection.objects.link(text_object)

        # Optionally, you can set active object
        context.view_layer.objects.active = text_object
        
        return {'FINISHED'}

    def invoke(self, context, event):
        # Show the file selector
        context.window_manager.fileselect_add(self)
        return {'RUNNING_MODAL'}

def menu_func_import(self, context):
    self.layout.operator(TextImport.bl_idname, text="Import Text (.md/.txt)")

def register():
    bpy.utils.register_class(TextImport)
    bpy.types.TOPBAR_MT_file_import.append(menu_func_import)

def unregister():
    bpy.types.TOPBAR_MT_file_import.remove(menu_func_import)
    bpy.utils.unregister_class(TextImport)

if __name__ == "__main__":
    register()

register_class(...):
Info: Registering operator class: 'TextImport', bl_idname 'curve.text_import' has been registered before, unregistering previous
register_class(...):


# Just Drag'nDrop

In [ ]:
import bpy


class TextImport(bpy.types.Operator):
    """Import a text file (.md or .txt) as a text object"""
    bl_idname = "curve.text_import"
    bl_label = "Import a text file"
    filepath: bpy.props.StringProperty(subtype='FILE_PATH', options={'SKIP_SAVE'})

    @classmethod
    def poll(cls, context):
        return context.area and context.area.type == "VIEW_3D"

    def execute(self, context):
        """Handles the import logic for .md and .txt files"""
        if not self.filepath or not self.filepath.endswith((".md", ".txt")):
            return {'CANCELLED'}

        try:
            with open(self.filepath, 'r', encoding='utf-8') as file:
                text_curve = bpy.data.curves.new(type="FONT", name="Text")
                text_curve.body = file.read()
                text_object = bpy.data.objects.new(name="Text", object_data=text_curve)
                bpy.context.scene.collection.objects.link(text_object)
            return {'FINISHED'}
        except Exception as e:
            self.report({'ERROR'}, f"Failed to import file: {e}")
            return {'CANCELLED'}

    def invoke(self, context, event):
        """Handles file selection or drag-and-drop"""
        if self.filepath:
            return self.execute(context)
        context.window_manager.fileselect_add(self)
        return {'RUNNING_MODAL'}


class FH_TextImport(bpy.types.FileHandler):
    """File handler for .md and .txt text file imports"""
    bl_idname = "curve.fh_text_import"
    bl_label = "File handler for text object import"
    bl_import_operator = "curve.text_import"
    bl_file_extensions = ".md;.txt"

    @classmethod
    def poll_drop(cls, context):
        return context.area and context.area.type == "VIEW_3D"

bpy.utils.register_class(TextImport)
bpy.utils.register_class(FH_TextImport)




# Combined

In [ ]:
import bpy

# -------------------------------------------------------------------
# Operator: Import Text/Markdown as a Text Object
# -------------------------------------------------------------------

class TextImport(bpy.types.Operator):
    """Import a text file (.md or .txt) as a Text Object"""
    bl_idname = "curve.text_import"
    bl_label = "Import Text (.md/.txt)"
    
    # Property to hold the file path (set by the file browser or drop event)
    filepath: bpy.props.StringProperty(
        subtype='FILE_PATH',
        options={'SKIP_SAVE'},
        default="",
        name="File Path"
    )
    # Used only when invoking from the file browser (menu import)
    filter_glob: bpy.props.StringProperty(
        default="*.txt;*.md",
        options={'HIDDEN'}
    )
    
    @classmethod
    def poll(cls, context):
        # Allow this operator in any context;
        # the file handler will restrict drag–n–drop to the 3D View.
        return True
    
    def execute(self, context):
        if not self.filepath or not self.filepath.lower().endswith((".md", ".txt")):
            self.report({'WARNING'}, "Please select a valid .txt or .md file.")
            return {'CANCELLED'}
        try:
            with open(self.filepath, 'r', encoding='utf-8') as file:
                text_content = file.read()
            
            # Create a new text curve (a Font-type curve)
            text_curve = bpy.data.curves.new(name="ImportedText", type="FONT")
            text_curve.body = text_content
            
            # Create a new object from that curve
            text_object = bpy.data.objects.new(name="ImportedText", object_data=text_curve)
            context.scene.collection.objects.link(text_object)
            
            # Optionally, make the new object active and selected
            context.view_layer.objects.active = text_object
            text_object.select_set(True)
            
            self.report({'INFO'}, "Text imported successfully.")
            return {'FINISHED'}
        except Exception as e:
            self.report({'ERROR'}, f"Failed to import file: {e}")
            return {'CANCELLED'}
    
    def invoke(self, context, event):
        # If the operator was called with a filepath already set (by a drag–n–drop), run immediately.
        if self.filepath:
            return self.execute(context)
        # Otherwise (from the menu), show the file selector.
        context.window_manager.fileselect_add(self)
        return {'RUNNING_MODAL'}


# -------------------------------------------------------------------
# File Handler: Enable Drag–n–Drop in the 3D View for .txt and .md Files
# -------------------------------------------------------------------

class FH_TextImport(bpy.types.FileHandler):
    """File handler for drag–n–drop text file (.md or .txt) imports"""
    bl_idname = "curve.fh_text_import"
    bl_label = "File handler for Text Import"
    bl_import_operator = "curve.text_import"
    bl_file_extensions = ".md;.txt"
    
    @classmethod
    def poll_drop(cls, context):
        # Only allow drops in the 3D View (you must drop onto a 3D View region)
        return context.area is not None and context.area.type == "VIEW_3D"


# -------------------------------------------------------------------
# Menu Registration: Add to the TOPBAR > File > Import menu
# -------------------------------------------------------------------

def menu_func_import(self, context):
    self.layout.operator(TextImport.bl_idname, text="Import Text (.md/.txt)")


# -------------------------------------------------------------------
# Registration
# -------------------------------------------------------------------

classes = (TextImport, FH_TextImport)

def register():
    for cls in classes:
        bpy.utils.register_class(cls)
    bpy.types.TOPBAR_MT_file_import.append(menu_func_import)

def unregister():
    bpy.types.TOPBAR_MT_file_import.remove(menu_func_import)
    for cls in reversed(classes):
        bpy.utils.unregister_class(cls)

if __name__ == "__main__":
    register()